In [1]:
remove globalmaxpooling, flatten
keras.reshape(55*991,256)
gru layer(units)


SyntaxError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path='/content/drive/MyDrive/Audio_Tagging_MLSP'
import numpy as np 
import os
import glob
import pandas as pd
import librosa
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from keras.preprocessing import image
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, hamming_loss, multilabel_confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
def to_multihot(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.delete(active_events, 8)

In [5]:
path='/content/drive/MyDrive/Audio_Tagging_MLSP'
X_train=np.load(os.path.join(path, "X_train.npy"))
X_test=np.load(os.path.join(path, "X_test.npy"))
Y_train=np.load(os.path.join(path, "Y_train.npy"))
Y_test=np.load(os.path.join(path, "Y_test.npy"))

In [6]:
X_train=X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_train.shape
X_test=X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))
X_test.shape

(2000, 64, 1000, 1)

In [12]:
def create_model(input_shape):
    model = tf.keras.Sequential()
    # 1st conv layer
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())

    #2nd conv layer
    model.add(tf.keras.layers.Conv2D(64, (2, 2), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.3))

    # 3rd conv layer
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.25))
        
    # 4th conv layer
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))
        
    # 5th conv layer
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())

    # CNN to CRNN
    model.add(tf.keras.layers.Reshape(target_shape=((55, 253696))))
    model.add(tf.keras.layers.GRU(128))
    
    #Dense Layer
    model.add(tf.keras.layers.Dense(256, activation='relu'))

    # output layer
    model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','AUC'])
    
    return model

In [10]:
path='/content/drive/MyDrive/Audio_Tagging_MLSP/Final_CRNN_model'
filepath=os.path.join(path, "./CRNN_Model.h5")
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_auc',
    mode='max',
    save_best_only=True)

In [13]:
model = create_model(input_shape=X_train.shape[1:])
# model = tf.keras.models.load_model('/content/drive/MyDrive/Audio_Tagging_MLSP/CNN_model.h5',custom_objects={"macro_f1":macro_f1})
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 62, 998, 32)       320       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 62, 998, 32)      0         
 2D)                                                             
                                                                 
 batch_normalization_5 (Batc  (None, 62, 998, 32)      128       
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 61, 997, 64)       8256      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 61, 997, 64)      0         
 2D)                                                             
                                                      

In [ ]:
history=model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=140, verbose = 2, callbacks=[model_checkpoint_callback])

Epoch 1/140
